<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/RTOFS/AWS_download/rtofs_cut_out_region.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def get_index(lat_array, lon_array, lat0, lon0):
  # First, find the index of the grid point nearest a specific lat/lon.
  abslat = np.abs(lat_array-lat0)
  abslon = np.abs(lon_array-lon0)
  c = np.maximum(abslon, abslat)

  ([xloc], [yloc]) = np.where(c == np.min(c))

  #point_ds = ds.sel(X=xloc, Y=yloc)
  #print(xloc, yloc)
  return [xloc, yloc]

In [ ]:
# download a data file to test

url="https://noaa-nws-rtofs-pds.s3.amazonaws.com/rtofs.20250301/rtofs_glo_2ds_n000_diag.nc"

fName_save="rtofs_glo_2ds_n_20250301_000_diag.nc"

print(url, fName_save)

!wget $url -O $fName_save

In [ ]:
ds = xr.open_dataset(fName_save)

In [ ]:
# To cut out a region:
# --------------------
#
#         x (P3) ---- x (P4)
#         |           |
#         x (P1) ---- x (P2)
#
#
#P1
[x1, y1] = get_index(ds.Latitude.values, ds.Longitude.values, -10, 100)
#P2
[x2, y2] = get_index(ds.Latitude.values, ds.Longitude.values, -10, 290)
# x1 will be same as x2
# ----------------------
#P3
[x3, y3] = get_index(ds.Latitude.values, ds.Longitude.values, 10, 100)
#P4
[x4, y4] = get_index(ds.Latitude.values, ds.Longitude.values, 10, 290)
# x3 will be same as x4
# ----------------------

print(f'Lower left [lon, lat]= {ds.Longitude.values[x1, y1]}, {ds.Latitude.values[x2, y2]}')

print(f'Upper right [lon, lat]: {ds.Longitude.values[x4, y4]}, {ds.Latitude.values[x4, y4]}')

print("\n--\n")
print(x1, y1, x2, y2)
print("\n--\n")
print(x3, y3, x4, y4)

In [ ]:
ds.sel(X=slice(1, 4500), Y=slice(1000, 2000)).ssh.plot()

plt.axvline(x=y1, color='k', ls='-')
plt.axvline(x=y2, color='k', ls='--')

plt.axhline(y=x1, color='b', ls='-')
plt.axhline(y=x3, color='b', ls='--')

## Check to make sure the cut-out region is correct.

In [ ]:
ds.sel(X=slice(y1, y2), Y=slice(x1, x3)).ssh.plot()

plt.plot(y1, x1, 'ko', ms=20)
plt.plot(y2, x3, 'bs', ms=20)
#
plt.plot(y2, x1, 'go', ms=20)
plt.plot(y1, x3, 'cs', ms=20)